# PROJETO DE DETECÇÃO DE FRAUDES EM COMPRAS ONLINE

FONTE: https://www.kaggle.com/datasets/ealaxi/paysim1?resource=download

IDEALIZADO EM: https://medium.com/coders-camp/230-machine-learning-projects-with-python-5d0c7abf8265

## IMPORT DE BIBLIOTECAS

### conectar com o google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### integrar spark no colab
https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/

In [8]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # download Java


In [19]:
!wget -q https://www-us.apache.org/dist/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.7.tgz #  install Apache Spark 3.0.1 with Hadoop 2.7


In [21]:
!tar xf spark-3.3.1-bin-hadoop3.tgz # unzip that folder


In [11]:
!pip install -q findspark # install findspark library

### iniciar ambiente spark

In [16]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

In [22]:
import findspark
findspark.init()

In [23]:
findspark.find()


'/content/spark-3.3.1-bin-hadoop3'

In [24]:
# import SparkSession from pyspark.sql and create a SparkSession, which is the entry point to Spark.
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [25]:
spark


## IMPORT DE FUNÇÕES

## IMPORT DE DADOS

In [30]:
df = spark.read.csv("/content/drive/MyDrive/..Python_Codes/Projeto_fraude_compras_online/content/PS_20174392719_1491204439457_log.csv", header=True, inferSchema=True)


In [40]:
df.show()
print(f"Esse dataframe tem {df.count()} linhas e {len(df.columns)} colunas")

+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|    type|   amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|   1| PAYMENT|  9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|
|   1| PAYMENT|  1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|             0|
|   1|TRANSFER|    181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|             0|
|   1|CASH_OUT|    181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|      1|             0|
|   1| PAYMENT| 11668.14|C2048537720|      41554.0|      29885.86|M123070170

## TRATAMENTO E LIMPEZA DE DADOS

## ANÁLISE

## PLOT DE RESULTADOS


## ATUAÇÃO COM OUTSIDERS